# 长期会话记忆

- `InMemoryChatMessageHistory` 仅能在内存中临时存储会话，程序退出则记忆丢失
- `FileChatMessageHistory` 可以将会话保存在文件中，程序退出后仍然保存

## FileChatMessageHistory 类实现思路

- 基于文件存储会话记录，以 `session_id` 为文件名，文件内容为会话记录
- 继承 `BaseChatMessageHistory` 类，实现如下方法：
  - `add_messages`：同步模式，添加消息
  - `messages`：同步模式，获取所有消息
  - `clear`：同步模式，清空所有消息
- 官方在 `BaseChatMessageHistory` 类的注释中提供了一个基于文件存储的示例代码

In [1]:
import os, json
from typing import Sequence

from langchain_community.chat_models import ChatTongyi
from langchain_core.messages import message_to_dict, messages_from_dict, BaseMessage
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableWithMessageHistory

In [4]:
# message_to_dict：单个消息对象（BaseMessage类实例） -> 字典
# messages_from_dict：[字典、字典...]  -> [消息、消息...]
# AIMessage、HumanMessage、SystemMessage 都是BaseMessage的子类

class FileChatMessageHistory(BaseChatMessageHistory):
    def __init__(self,session_id,storage_path):
        self.session_id = session_id        # 会话id
        self.storage_path = storage_path    # 不同会话id的存储文件，所在的文件夹路径
        # 完整的文件路径
        self.file_path = os.path.join(self.storage_path, self.session_id)

        # 确保文件夹是存在的
        os.makedirs(os.path.dirname(self.file_path), exist_ok=True)

    @property
    def messages(self) -> list[BaseMessage]:
        try:
            with open(self.file_path,"r",encoding="utf-8") as f:
                messages_data = json.load(f)
            return messages_from_dict(messages_data)
        except FileNotFoundError:
            return []

    def add_messages(self, messages: Sequence[BaseMessage]) -> None:
        all_messages = list(self.messages)  # 已有的消息列表
        all_messages.extend(messages)       # 新的和已有的融合成一个list

        new_messages = [message_to_dict(message) for message in all_messages]

        # 将数据写入文件
        with open(self.file_path, "w", encoding="utf-8") as f:
            json.dump(new_messages, f)

    def clear(self) -> None:
        os.makedirs(os.path.dirname(self.file_path), exist_ok=True)
        with open(self.file_path, "w", encoding="utf-8") as f:
            json.dump([], f)

In [6]:
model = ChatTongyi(model="qwen3-max")

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你需要根据会话历史回应用户问题。对话历史："),
        MessagesPlaceholder("chat_history"),
        ("human", "请回答如下问题：{input}")
    ]
)

In [7]:
str_parser = StrOutputParser()

def print_prompt(full_prompt):
    print("="*20)
    print(full_prompt.to_string())
    print("="*20)
    return full_prompt

In [8]:
base_chain = prompt | print_prompt | model | str_parser

def get_history(session_id):
    return FileChatMessageHistory(session_id, "./chat_history")

In [9]:
# 创建一个新的链，对原有链增强功能：自动附加历史消息
conversation_chain = RunnableWithMessageHistory(
    base_chain,     # 被增强的原有chain
    get_history,    # 通过会话id获取InMemoryChatMessageHistory类对象
    input_messages_key="input",             # 表示用户输入在模板中的占位符
    history_messages_key="chat_history"     # 表示用户输入在模板中的占位符
)

In [10]:
# 固定格式，添加LangChain的配置，为当前程序配置所属的session_id
session_config = {
    "configurable": {
        "session_id": "user_001"
    }
}

res = conversation_chain.invoke({"input": "小明有2个猫"}, session_config)
print("第1次执行：", res)
#
res = conversation_chain.invoke({"input": "小刚有1只狗"}, session_config)
print("第2次执行：", res)

res = conversation_chain.invoke({"input": "总共有几个宠物"}, session_config)
print("第3次执行：", res)

System: 你需要根据会话历史回应用户问题。对话历史：
Human: 请回答如下问题：小明有2个猫
第1次执行： 小明有2只猫。请问您想了解什么相关信息呢？比如猫的名字、品种，还是与小明和猫有关的其他问题？
System: 你需要根据会话历史回应用户问题。对话历史：
Human: 小明有2个猫
AI: 小明有2只猫。请问您想了解什么相关信息呢？比如猫的名字、品种，还是与小明和猫有关的其他问题？
Human: 请回答如下问题：小刚有1只狗
第2次执行： 小刚有1只狗。
System: 你需要根据会话历史回应用户问题。对话历史：
Human: 小明有2个猫
AI: 小明有2只猫。请问您想了解什么相关信息呢？比如猫的名字、品种，还是与小明和猫有关的其他问题？
Human: 小刚有1只狗
AI: 小刚有1只狗。
Human: 请回答如下问题：总共有几个宠物
第3次执行： 小明有2只猫，小刚有1只狗，所以他们一共有：

2（猫） + 1（狗） = **3只宠物**。

答：总共有 **3** 个宠物。
